# Demo of JSON tool response processing using code generation
This Jupyter notebook is a demo of the JSON processor component.

This approach is effective when JSON tool responses are large and complex/highly nested and when the processing required is not trivial.

Up to 20% improvement in accuracy on some queries (over prompting a model directly to extract the answer) even when using frontier models like GPT-4o.

In [ ]:
from typing import cast
import json
from altk.core.toolkit import AgentPhase
from altk.core.llm import get_llm
from altk.post_tool.code_generation.code_generation import (
    CodeGenerationComponent,
    CodeGenerationComponentConfig,
)
from altk.post_tool.core.toolkit import (
    CodeGenerationRunInput,
    CodeGenerationRunOutput,
)

The inputs to this component are: a user query (`user_query`) and an API response as a JSON object (`api_response`)

In [ ]:
user_query = (
    "Which hotels have free parking? Output a comma separated list of hotel names."
)

with open("codegen_long_response_example.json", "r") as file:
    response = json.load(file)

print(response)

Configure the LLM you will use to generate the Python code that will process the JSON response. 

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your_api_key_here"

OPENAI_CLIENT = get_llm("openai.sync")  # ValidatingLLMClient
config = CodeGenerationComponentConfig(
    llm_client=OPENAI_CLIENT(
        model_name="o4-mini",
    )
)

Initialize the JSON processor component

In [ ]:
component = CodeGenerationComponent(config=config)

The user query and API response is passed to the LLM model to generate the Python code. The code is executed to obtain the final answer from the API response that addresses the user's query.

In [ ]:
input_data = CodeGenerationRunInput(
    messages=[], nl_query=user_query, tool_response=response
)

Run the component and generate the output. 

In [ ]:
output = cast(
    CodeGenerationRunOutput, component.process(input_data, AgentPhase.RUNTIME)
)
print(output)